# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import sklearn
import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Tensorflow Models

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def create_placeholders_rnn(num_features, sequence_length, output_shape):
    X = tf.placeholder(tf.float32, shape=(None,sequence_length,num_features), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y


#regularizers.l2(l2_lambda_regularization)
def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

def tf_lstm_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(20)
    outputs,states = tf.contrib.rnn.static_rnn(lstm_cell, X, dtype=tf.float32)
    
    A2 = tf.layers.dense(outputs[-1], 10, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    return y 
    

In [4]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        
        for weight in weights:
        
            print(weight.shape)
            print(weight)

# Create the Data Handler

In [5]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [6]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [7]:
def tf_compiled_model(num_features, sequence_length, output_shape):
    
    tf.reset_default_graph()
    X, y = create_placeholders_rnn(num_features, sequence_length, output_shape)
    
    X = tf.unstack(X, axis=1)
    
    y_pred = tf_lstm_model(X)
    cost = tf.losses.mean_squared_error(y, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.5).minimize(total_cost)
    
    return {'X_placeholder':X, 'y_placeholder':y, 'y_pred':y_pred, 'cost':cost, 'total_cost':total_cost, 'optimizer':optimizer}

In [8]:

tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', None, lib_type='tensorflow', data_handler=dHandler_cmaps)


# Load Data

In [9]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [10]:
# #Load Non sequenced data (unroll sequence into a single feature vector)

# tModel.data_handler.data_scaler = None
# tModel.data_scaler = min_max_scaler

# tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
# #tModel.data_handler.print_data()
# tModel.print_data()

In [10]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()


#tModel.print_data()

X_batches, y_batches, total_batch = CMAPSAuxFunctions.get_minibatches(tModel.X_train,tModel.y_train, 512)
print(X_batches[0].shape)
X_input = tf.unstack(X_batches[0], axis=1)
print(len(X_input))
print(X_input[0].shape)

print("y_temp")
print(len(y_batches))
print(y_batches[0].shape)

"""
#x_temp, y_temp = sklearn.utils.shuffle(tModel.X_train,tModel.y_train)
print("x_temp")
print(len(X_batches))
print(X_batches[0].shape)
print("y_temp")
print(len(y_batches))
print(y_batches[0].shape)
"""

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
(512, 24, 14)
24
(512, 14)
y_temp
36
(512, 1)


'\n#x_temp, y_temp = sklearn.utils.shuffle(tModel.X_train,tModel.y_train)\nprint("x_temp")\nprint(len(X_batches))\nprint(X_batches[0].shape)\nprint("y_temp")\nprint(len(y_batches))\nprint(y_batches[0].shape)\n'

# Train the model and test some Tunable Model functionalities

In [11]:
nFeatures = len(selected_features)

#lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

# shape = len(selected_features)*tModel.data_handler.sequence_length
# print(shape)

shape = (nFeatures,tModel.data_handler.sequence_length)

#modelRULSN = RULmodel_SN_5(shape)
#modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#model = get_compiled_model(shape, model_type='ann')
#tModel.change_model('ModelRUL_SN_1', model, 'keras')

model = tf_compiled_model(nFeatures,tModel.data_handler.sequence_length, 1)
tModel.change_model('ModelRUL_LSTM_1', model, 'tensorflow')

#tModel.print_data()

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

#print("Printing model weights")
#print_model_weights(tModel.model)

tModel.epochs = 200

sess = tf.Session()

tModel.train_model(tf_session=sess, verbose=1)

TypeError: unhashable type: 'list'

In [ ]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, tf_session=sess)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))